# Speech Understanding 
# Lecture 14: Personal Assistant


### Mark Hasegawa-Johnson, KCGI

1. <a href="#section1">What time is it?</a>
1. <a href="#section2">Tell me a joke!</a>
1. <a href="#section3">Show me my calendar for today</a>
1. <a href="#section4">Personal assistant</a>
1. <a href="#homework">Homework</a>


<a id='section1'></a>

## 1. What time is it?

We will use the `datetime` package to find out what time it is.  We will get the date and time in ISO standard format, then parse them, and read the result back to the user.  First, let's see what the ISO standard format looks like.

In [1]:
import datetime

print(datetime.datetime.now().isoformat())

2024-07-17T10:58:05.565788


You can see that the ISO standard format has the date, followed by the the hour, minutes, and seconds.  Our personal assistant will only read to us the hour and minutes.  We can use `split` to split the string at the `T` character, and then to split it at every `:` character, then read out only the relevant part:


In [2]:
(date, time) = datetime.datetime.now().isoformat().split("T")

(hour, minutes, seconds) = time.split(":")

print(hour+"時"+minutes+"分です")

10時58分です


Let's create a function that tells us the current time.

In [3]:
import datetime, gtts

def what_time_is_it(lang, filename):
    '''
    Tell me what time it is.
    
    Parameters:
    lang (str) - language in which to speak
    filename (str) - the filename into which the audio should be recorded
    '''
    (date, time) = datetime.datetime.now().isoformat().split("T")
    (hour, minutes, seconds) = time.split(":")
    if lang=="en":
        text = hour+" hours and "+minutes+" minutes"
    elif lang=="ja":
        text = hour+"時"+minutes+"分です"
    elif lang=="zh":
        text = "现在是"+hour+"点"+"分"
    else:
        text="I'm sorry, I don't know that language"
    gtts.gTTS(text,lang=lang).save(filename)
 

If you've created that file, you can now test it by running the following block:

In [ ]:
what_time_is_it("it", "time.mp3")
import librosa, IPython
x, fs = librosa.load('time.mp3')
IPython.display.Audio(data=x, rate=fs)

<a id='section2'></a>

## 2. Tell me a joke!

If you want your personal assistant to tell jokes, you need a good source of jokes.  There are many lists of jokes on the internet, though most of the jokes are not very funny.

### 2.1 Jokes in English

In English, let's use this list of jokes from the `yesinteractive/dadjokes` app:  https://raw.githubusercontent.com/yesinteractive/dadjokes/master/controllers/jokes.txt

In [ ]:
import requests
req = requests.get("https://raw.githubusercontent.com/yesinteractive/dadjokes/master/controllers/jokes.txt")

texts = {}
texts['en'] = req.text
with open('jokes_en.txt','w') as f:
    f.write(texts['en'].replace('<>','  '))
    
jokes = {}
jokes['en'] = []
with open('jokes_en.txt', 'r') as f:
    jokes['en'] = f.readlines()

for n in range(6):
    print(jokes['en'][n])

### 2.2 Jokes in Japanese

Here is a web page containing some jokes in Japanese: http://www.gujo-tv.ne.jp/~circleband/oyaji%20gag.htm.  

Open the page source, and look at `content` at the top.  Notice that this file is stored in `Shift_JIS`, which is a pre-unicode encoding for Japanese characters.  So the first thing we need to do is to convert it to unicode.  One way to do that is by saving the raw binary to a file, and then reading it back in using the `Shift_JIS` decoder:


In [ ]:
import requests

rawbinary = requests.get("http://www.gujo-tv.ne.jp/~circleband/oyaji%20gag.htm").content
with open('jokes_ja.htm','wb') as f:
    f.write(rawbinary)
    
with open('jokes_ja.htm', 'r', encoding='shiftjis') as f:
    texts['ja'] = f.read()

print(texts['ja'][0:1000])


Notice that each joke is the third `<td>` tag under a `<tr>` tag.  Let's use BeautifulSoup to find those.


In [ ]:
import bs4
soup = bs4.BeautifulSoup(texts['ja'], "html.parser")

jokes['ja'] = []
for tr in soup.find_all('tr'):
    tdlist = tr.find_all('td')
    jokes['ja'].append(tdlist[2].text + '\n')

for n in range(5):
    print(jokes['ja'][n])

with open('jokes_ja.txt','w') as f:
    f.writelines(jokes['ja'])

### 2.3 Jokes in Chinese

As in Japanese, we need to somehow extract jokes from a web page.  Let's try this web page:  http://www.ziyexing.com/files_8/xiaohua_01.htm.  This file is encoded in gb2312, so we will have to convert it to Unicode, just as we did with the Japanese jokes.

In [ ]:
import requests

rawbinary = requests.get("http://www.ziyexing.com/files_8/xiaohua_01.htm").content
    
with open('jokes_zh.htm','wb') as f:
    f.write(rawbinary)
    
with open('jokes_zh.htm', 'r', encoding="gb2312", errors="ignore") as f:
    texts['zh'] = f.read()

print(texts['zh'][:2000])

* All of the jokes are in one long paragraph.  This paragraph is distinguished from other paragraphs only because it has `style="line-height: 150%"`
* Each joke begins with a line in boldface (`<b>...</b>`), starting with the character `○`.  We can probably just discard the `<b>` and `</b>` tags using the python string `replace` function, and then split the paragraph at the `○` character.
* Each joke contains several lines, with lots of extra whitespace.  We can split on `\n` characters, eliminate `<br>` tags, and then use `strip` to get rid of extra whitespace.


In [ ]:
import bs4
soup = bs4.BeautifulSoup(texts['zh'], "html.parser")

ptag = soup.find("p",{'style':'line-height: 150%'})
jokestext = ptag.text.replace('</b>','').replace('<br>','').replace('<b>','')

jokes['zh'] = []
for rawjoke in jokestext.split('○'):
    lines = rawjoke.split('\n')
    joke = lines[0].strip() + ': '
    if len(lines) > 1:
        for line in lines[1:]:
            joke += line.strip()
        jokes['zh'].append(joke + '\n')

for n in range(10):
    print('Joke number ',n,': ', jokes['zh'][n])

with open('jokes_zh.txt','w') as f:
    f.writelines(jokes['zh'])

### 2.4 Tell me a joke!

Let's ask gtts to tell us a joke chosen at random from one of these lists.  Add the following to your `week14.py` file:

In [ ]:
import gtts, bs4, random

def tell_me_a_joke(lang, audiofile):
    '''
    Tell me a joke.
    
    @params:
    filename (str) - filename containing the database of jokes
    lang (str) - language
    audiofile (str) - audiofile in which to record the joke
    '''
    filename = 'jokes_%s.txt'%(lang)
    with open(filename) as f:
        jokes = f.readlines()
    joke = random.choice(jokes)
    print(joke.strip())
    gtts.gTTS(joke.strip(), lang=lang).save(audiofile)


Let's try it:

In [ ]:
tell_me_a_joke("zh", 'joke.mp3')

import librosa, IPython
x, fs = librosa.load('joke.mp3')
IPython.display.Audio(data=x, rate=fs)

<a id='section3'></a>

## 3. What day is it today?

Let's ask our assistant to tell us today's date.  In order to give her something to say, we will also ask her to say what is today's current date, using the python `datetime` package.  
`datetime` gives the current date in several different ways.  For example,

In [ ]:
import datetime
print('The year is',datetime.date.today().year)
print('The month is',datetime.date.today().month)
print('The day is',datetime.date.today().day)
print('It is the',datetime.date.today().isoweekday(),'day of the week')

We can use some dictionaries to make those numbers more friendly.

In [ ]:
def what_day_is_it(lang, audiofile):
    '''
    Tell me what day it is.

    @params:
    lang (str) - language in which to record the date
    audiofile (str) - filename in which to read the date
    
    @returns:
    url (str) - URL that you can look up in order to see the calendar for this month and year
    '''
    today = datetime.date.today()
    year = today.year
    month = today.month
    day = today.day
    weekday = today.isoweekday()
    if lang=="en":
        weekdays=['','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
        months=['','January','February','March','April','May','June','July','August','September','October','November','December']
        text = "%s, %s %d, %d"%(weekdays[weekday],months[month],day,year)
        gtts.gTTS("Today is "+text,lang="en").save(audiofile)
    elif lang=="ja":
        weekdays=' 月火水木金土日'
        text="%s曜日,%d月%d日, %d年"%(weekdays[weekday],month,day,year)
        gtts.gTTS("今日は"+text,lang="ja").save(audiofile)
    elif lang=="zh":
        weekdays=['','周一','周二','周三','周四','周五','周六','星期日']
        text='%s, %d月%d日, %d年'%(weekdays[weekday],month,day,year)
        gtts.gTTS("今天是"+text,lang="zh").save(audiofile)


Let's try it:

In [ ]:
what_day_is_it("en","date.mp3")

import librosa, IPython
x, fs = librosa.load('date.mp3')
IPython.display.Audio(data=x, rate=fs)

<a id='section4'></a>

## 4. Personal assistant

Now let's put it all together in a personal assistant app.  Your personal assistant will listen to you, and respond when you make any of the following four types of requests:

* If you say anything containing "What time," it will tell you the time
* If you say anythin containing the word "joke," it will tell you a joke
* If you say anything containing the word "calendar," it will open your calendar
* If you say something else, it will say "I'm sorry, I didn't understand you!"

First, let's create a convenience function that listens to us:

In [ ]:
import speech_recognition

def personal_assistant(lang, filename):
    if lang=="en":
        keywords = ["what time", "joke", "what day", "I'm sorry, I didn't understand you"]
    elif lang=="ja":
        keywords = ["何時","冗談","何日","すみません、よくわかりませんでした"]
    elif lang=="zh":
        keywords = ["几奌","玩笑","什么日子","对不起，我没听懂你的话"]
    else:
        speech_package.synthesize("I don't know that language!","en",filename)
        return

    text = input("What would you like me to do?")
            
    print("I heard",text)
    if keywords[0] in text:
        what_time_is_it(lang, filename)
    elif keywords[1] in text:
        tell_me_a_joke(lang, filename)
    elif keywords[2] in text:
        what_day_is_it(lang, filename)
    else:
        print(keywords[3])
        print('I will try again')
            

Now let's try running the personal assistant:

In [ ]:
personal_assistant("en", "test.mp3")

x, fs = librosa.load("test.mp3")
IPython.display.Audio(data=x, rate=fs)

<a id='homework'></a>

## Homework

In the homework file `homework14.py`, please create four functions `what_time_is_it`, `tell_me_a_joke`, `what_day_is_it`, and `personal_assistant` like those above.

In [ ]:
import homework14, importlib
importlib.reload(homework14)
help(homework14.what_time_is_it)

In [ ]:
help(homework14.tell_me_a_joke)

In [ ]:
help(homework14.what_day_is_it)

In [ ]:
help(homework14.personal_assistant)

When everything is working, you should be able to run personal_assistant from your homework file:

In [ ]:
importlib.reload(homework14)
homework14.personal_assistant('ja','test.mp3')
x, fs = librosa.load("test.mp3")
IPython.display.Audio(data=x, rate=fs)

### Receiving your grade

In order to receive a grade for your homework, you need to:

1. Run the following code block on your machine.  The result may list some errors, and then in the very last line, it will show a score.  That score (between 0% and 100%) is the grade you have earned so far.  If you want to earn a higher grade, please continue editing `homework3.py`, and then run this code block again.
1. When you are happy with your score (e.g., when it reaches 100%), choose `File` $\Rightarrow$ `Save and Checkpoint`.  Then use `GitHub Desktop` to commit and push your changes.
1. Make sure that the 100% shows on your github repo on github.com.  If it doesn't, you will not receive credit.

In [ ]:
import importlib, grade
importlib.reload(grade)